In [ ]:
import pandas as pd

df = pd.read_csv("Dataframe/combined_data.csv")

df.head()


In [ ]:
# # Rename the columns needed
df_us_20208_summary = df_us_20208.rename(columns={
                                        "D1_A": "AGE",
                                        "D1_G": "GENDER",
                                        "VAXANY": "VACCINE STATUS",
                                        "L7": "TRUST",
                                        "Xp6": "POS-CASES",
                                        "CVAX4": "TRUST GOV",
                                        "D0": "HEALTHY",
                                        "D2": "STATE",
                                        "D3": "RURAL",
                                        "D4": "INCOME",
                                        "D5_E": "HISPANIC",
                                        "D5_M": "RACE",
                                        "D6": "RELIGION",
                                        "D6_E": "RELIGIOUS",
                                        "D7": "HEALTH COVERAGE",
                                        "D8": "POLITICS",
                                        "D9": "WORK STATUS",
                                        "D13": "ILLNESS",
                                        "D15": "EDUCATION",
                                         })